# Part 1 - Evaluate simulations for some general correlations and scalings

### First load some references

In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases.PrintingNip;
Init();

### Init Database etc.

In [ ]:
string ProjectName = "SFB1194_K65_Part1";

In [ ]:
BoSSSshell.WorkflowMgm.Init(ProjectName);

In [ ]:
static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

### Load Simulations

In [ ]:
var sessions = myDb.Sessions.Where(s => s.ProjectName == ProjectName).ToArray(); // theoretically this database contains only projects with the correct project name

In [ ]:
sessions.Count()

### Setup Post-Processing

In [ ]:
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Clear();

In [ ]:
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("PressureRange", Postprocessing.PressureRange);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("VelocityXMax", Postprocessing.VelocityXMax);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("Massflux", Postprocessing.Massflux);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("NipShearRate", Postprocessing.NipShearRate);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("NipShearRateCylinder", Postprocessing.NipShearRateCylinder);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("ShearStressCylinder", Postprocessing.ShearStressCylinder);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("NipShearRateSubstrate", Postprocessing.NipShearRateSubstrate);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("ShearStressSubstrate", Postprocessing.ShearStressSubstrate);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("ViscousDissipation", Postprocessing.ViscousDissipation);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("PositionOfSynchronousFlow", Postprocessing.PositionOfSynchronousFlow);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("PositionOfStagnatingFlow", Postprocessing.PositionOfStagnatingFlow);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatSynchronousPoint", Postprocessing.dPdXatSynchronousPoint);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatStagnationPoint", Postprocessing.dPdXatStagnationPoint);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatNip", Postprocessing.dPdXatNip);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatConstantX", Postprocessing.dPdXatConstantX);

### Create Datatable

In [ ]:
DataTable tab;
if(!File.Exists("./PrintingNip/Files/PrintingNip-Part1Export.csv")){
    tab = sessions.GetSessionTable(BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Select(kv => new Tuple<string, Func<ISessionInfo, object>>(kv.Key, kv.Value)).ToArray());
    tab.ToCSVFile("./PrintingNip/Files/PrintingNip-Part1Export.csv", ColSep: '%');
} else{
    tab = TableExtensions.FromCSVFile("../PrintingNip/Files/PrintingNip-Part1Export.csv", ColSep: '%');
}


### Export Datatable

In [ ]:
string[] Columns = new string[] {"PressureRange", "VelocityXMax", "Massflux", "NipShearRate", "NipShearRateCylinder", "ShearStressCylinder", "NipShearRateSubstrate", "ShearStressSubstrate", "ViscousDissipation", "PositionOfSynchronousFlow",  "PositionOfStagnatingFlow", "dPdXatSynchronousPoint", "dPdXatStagnationPoint", "dPdXatNip", "dPdXatConstantX"};
string[] Labels = new string[] {"Pressure", "Velocity", "Massflux", "Shearrate (Nip)", "Shearrate (Cylinder)", "Shearforce (Cylinder)", "Shearrate (Substrate)", "Shearforce (Substrate)", "Dissipation", "$x_{sync}$", "$x_{stag}$", @"$\\\frac{\\\partial p}{\\\partial x}_{sync}$", @"$\\\frac{\\\partial p}{\\\partial x}_{stag}$", @"$\\\frac{\\\partial p}{\\\partial x}_{nip}$", @"$\\\frac{\\\partial p}{\\\partial x}$"};
string[] Units = new string[] {@"$\\\left[\\\frac{N}{m^2}\\right]$", @"$\\\left[\\\frac{m}{s}\\right]$", @"$\\\left[\\\frac{kg}{s}\\right]$", @"$\\\left[\\\frac{1}{s}\\right]$", @"$\\\left[\\\frac{1}{s}\\right]$", @"$\\\left[N\\right]$", @"$\\\left[\\\frac{1}{s}\\right]$", @"$\\\left[N\\right]$", @"$\\\left[W\\right]$", @"$\\\left[m\\right]$", @"$\\\left[m\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$"}; // some escape character nonsense

In [ ]:
var tab2export = tab.ExtractColumns(Columns.ToList().Prepend("id:V_Wall").Prepend("id:P_Diff").Prepend("id:Radius").Prepend("id:delta").Prepend("SessionName").ToArray());
tab2export.ToHTMLFile("PrintingNip-Part1.html", Path.GetFullPath("./PrintingNip/Files"));
tab2export.ToCSVFile("./PrintingNip/Files/PrintingNip-Part1.csv", ColSep: ';');

### Plot Correlation dp = 0, V = const, h variable

In [ ]:
var tab2plt = tab.ExtractRows((i,row) => Convert.ToDouble(row["id:P_Diff"]).ApproxEqual(0.0) && Convert.ToDouble(row["id:V_Wall"])>0.0);

In [ ]:
List<Plot2Ddata> plts = new List<Plot2Ddata>();
foreach(string col in Columns){
    int k = Columns.IndexOf(col);
    var plt = tab2plt.ToPlot("id:delta", col, ColName_GroupSelection: new string[] { "id:V_Wall" });    
    plt.LogX   = true;
    plt.LogY   = true;
    plt.Xlabel = @"Nip Width $\\\left[m\\right]$";
    plt.Ylabel = Labels[k] + " " + Units[k];
    plt.LabelTitleFont = 32;
    plt.LabelFont = 24;
    plt.LegendFont = 18;
    plt.lmargin = 12;
    plt.bmargin = 4;
    plt.ShowLegend = true;    
    plt.LegendBox = true;
    if(plt.Regression().Average(kv => kv.Value) > 0){
        plt.LegendAlignment = new string[]{"i", "l", "t"};
        plt.LegendSwap = true;    
    } else{
        plt.LegendAlignment = new string[]{"i", "r", "t"};
        plt.LegendSwap = false;    
    }
    for(int n= 0; n < plt.dataGroups.Count(); n++){
        plt.dataGroups[n].Format.LineWidth = 2;
        plt.dataGroups[n].Format.PointSize = 1;
        plt.dataGroups[n].Name = "$V_W = " + Convert.ToDouble(plt.dataGroups[n].Name.Split("id:V_Wall").Last()).ToString("N1") + @" \\\frac{m}{s}$";
    }
    plt.ModFormat();
    plts.Add(plt);
}

In [ ]:
int k = 0;
foreach(var plt in plts){
    var gp = new Gnuplot();
    gp.PlotLogSlope(plt, format: new PlotFormat(lineColor: (LineColors)7));
    gp.Cmd("set key box width -5");
    plt.ToGnuplot(gp);
    gp.PlotCairolatex().SaveTo("./PrintingNip/Figures/Part1/P0_V_"+Columns[k]+".tex");
    k++;
}

### Plot Correlation dp = 0, V = variable, h const

In [ ]:
var tab2plt = tab.ExtractRows((i,row) => Convert.ToDouble(row["id:P_Diff"]).ApproxEqual(0.0)  && Convert.ToDouble(row["id:V_Wall"])>0.0);

In [ ]:
List<Plot2Ddata> plts = new List<Plot2Ddata>();
foreach(string col in Columns){
    int k = Columns.IndexOf(col);
    var plt = tab2plt.ToPlot("id:V_Wall", col, ColName_GroupSelection: new string[] { "id:delta" });    
    plt.LogX   = true;
    plt.LogY   = true;
    plt.Xlabel = @"Printing Velocity $\\\left[\\\frac{m}{s}\\right]$";
    plt.Ylabel = Labels[k] + " " + Units[k];
    plt.LabelTitleFont = 32;
    plt.LabelFont = 24;
    plt.LegendFont = 18;
    plt.lmargin = 12;
    plt.bmargin = 4;
    plt.ShowLegend = true;    
    plt.LegendBox = true;
    if(plt.Regression().Average(kv => kv.Value) > 0){
        plt.LegendAlignment = new string[]{"i", "l", "t"};
        plt.LegendSwap = true;    
    } else{
        plt.LegendAlignment = new string[]{"i", "r", "t"};
        plt.LegendSwap = false;    
    }
    for(int n= 0; n < plt.dataGroups.Count(); n++){
        plt.dataGroups[n].Format.LineWidth = 2;
        plt.dataGroups[n].Format.PointSize = 1;
        plt.dataGroups[n].Name = @"$\\\varepsilon = " + Convert.ToDouble(plt.dataGroups[n].Name.Split("id:delta").Last()).ToString("0e0").Replace("e","\\\\times 10^{") + "}" + @" m$";
    }
    plt.ModFormat();
    plts.Add(plt);
}

In [ ]:
int k = 0;
foreach(var plt in plts){
    var gp = new Gnuplot();
    gp.PlotLogSlope(plt, format: new PlotFormat(lineColor: (LineColors)7));
    gp.Cmd("set key box width -5");
    plt.ToGnuplot(gp);
    gp.PlotCairolatex().SaveTo("./PrintingNip/Figures/Part1/P0_H_"+Columns[k]+".tex");
    k++;
}

### Plot Correlation dp = const, V = 0, h variable

In [ ]:
var tab2plt = tab.ExtractRows((i,row) => Convert.ToDouble(row["id:V_Wall"]).ApproxEqual(0.0) && Convert.ToDouble(row["id:P_Diff"]) > 0.0);

with only pressure b.c. there is no stagnating point ...

In [ ]:
string[] Columns = new string[] {"PressureRange", "VelocityXMax", "Massflux", "NipShearRate", "NipShearRateCylinder", "ShearStressCylinder", "NipShearRateSubstrate", "ShearStressSubstrate", "ViscousDissipation", "dPdXatNip", "dPdXatConstantX"};
string[] Labels = new string[] {"Pressure", "Velocity", "Massflux", "Shearrate (Nip)", "Shearrate (Cylinder)", "Shearforce (Cylinder)", "Shearrate (Substrate)", "Shearforce (Substrate)", "Dissipation", @"$\\\frac{\\\partial p}{\\\partial x}_{nip}$", @"$\\\frac{\\\partial p}{\\\partial x}$"};
string[] Units = new string[] {@"$\\\left[\\\frac{N}{m^2}\\right]$", @"$\\\left[\\\frac{m}{s}\\right]$", @"$\\\left[\\\frac{kg}{s}\\right]$", @"$\\\left[\\\frac{1}{s}\\right]$", @"$\\\left[\\\frac{1}{s}\\right]$", @"$\\\left[N\\right]$", @"$\\\left[\\\frac{1}{s}\\right]$", @"$\\\left[N\\right]$", @"$\\\left[W\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$"}; // some escape character nonsense

In [ ]:
List<Plot2Ddata> plts = new List<Plot2Ddata>();
foreach(string col in Columns){
    int k = Columns.IndexOf(col);
    var plt = tab2plt.ToPlot("id:delta", col, ColName_GroupSelection: new string[] { "id:P_Diff" });    
    plt.LogX   = true;
    plt.LogY   = true;
    plt.Xlabel = @"Nip Width $\\\left[m\\right]$";
    plt.Ylabel = Labels[k] + " " + Units[k];
    plt.LabelTitleFont = 32;
    plt.LabelFont = 24;
    plt.LegendFont = 18;
    plt.lmargin = 12;
    plt.bmargin = 4;
    plt.ShowLegend = true;    
    plt.LegendBox = true;
    if(plt.Regression().Average(kv => kv.Value) > 0){
        plt.LegendAlignment = new string[]{"i", "l", "t"};
        plt.LegendSwap = true;    
    } else{
        plt.LegendAlignment = new string[]{"i", "r", "t"};
        plt.LegendSwap = false;    
    } 
    for(int n= 0; n < plt.dataGroups.Count(); n++){
        plt.dataGroups[n].Format.LineWidth = 2;
        plt.dataGroups[n].Format.PointSize = 1;
        plt.dataGroups[n].Name = @"$\\\Delta{p} = " + Convert.ToDouble(plt.dataGroups[n].Name.Split("id:P_Diff").Last()).ToString("0e0").Replace("e","\\\\times 10^{") + "}" + @" \\\frac{N}{m^2}$";
    }
    plt.ModFormat();
    plts.Add(plt);
}

In [ ]:
int k = 0;
foreach(var plt in plts){
    var gp = new Gnuplot();
    gp.PlotLogSlope(plt, format: new PlotFormat(lineColor: (LineColors)7));
    gp.Cmd("set key box width -7");
    plt.ToGnuplot(gp);
    gp.PlotCairolatex().SaveTo("./PrintingNip/Figures/Part1/V0_P_"+Columns[k]+".tex");
    k++;
}

### Plot Correlation dp = variable, V = 0, h const

In [ ]:
var tab2plt = tab.ExtractRows((i,row) => Convert.ToDouble(row["id:V_Wall"]).ApproxEqual(0.0) && Convert.ToDouble(row["id:P_Diff"])>0.0);

In [ ]:
List<Plot2Ddata> plts = new List<Plot2Ddata>();
foreach(string col in Columns){
    int k = Columns.IndexOf(col);
    var plt = tab2plt.ToPlot("id:P_Diff", col, ColName_GroupSelection: new string[] { "id:delta" });    
    plt.LogX   = true;
    plt.LogY   = true;
    plt.Xlabel = @"Pressure Difference $\\\left[\\\frac{N}{m^2}\\right]$";
    plt.Ylabel = Labels[k] + " " + Units[k];
    plt.LabelTitleFont = 32;
    plt.LabelFont = 24;
    plt.LegendFont = 18;
    plt.lmargin = 12;
    plt.bmargin = 4;
    plt.ShowLegend = true;    
    plt.LegendBox = true;
    if(plt.Regression().Average(kv => kv.Value) > 0){
        plt.LegendAlignment = new string[]{"i", "l", "t"};
        plt.LegendSwap = true;    
    } else{
        plt.LegendAlignment = new string[]{"i", "r", "t"};
        plt.LegendSwap = false;    
    }  
    for(int n= 0; n < plt.dataGroups.Count(); n++){
        plt.dataGroups[n].Format.LineWidth = 2;
        plt.dataGroups[n].Format.PointSize = 1;
        plt.dataGroups[n].Name = @"$\\\varepsilon = " + Convert.ToDouble(plt.dataGroups[n].Name.Split("id:delta").Last()).ToString("0e0").Replace("e","\\\\times 10^{") + "}" + @" m$";
    }
    plt.ModFormat();
    plts.Add(plt);
}

In [ ]:
int k = 0;
foreach(var plt in plts){
    var gp = new Gnuplot();
    gp.PlotLogSlope(plt, format: new PlotFormat(lineColor: (LineColors)7));
    gp.Cmd("set key box width -5");
    plt.ToGnuplot(gp);
    gp.PlotCairolatex().SaveTo("./PrintingNip/Figures/Part1/V0_H_"+Columns[k]+".tex");
    k++;
}

### Multiplots Correlation dp = variable, V = 0, h const and dp = 0, V = variable, h const

In [ ]:
var tab2pltL = tab.ExtractRows((i,row) => Convert.ToDouble(row["id:P_Diff"]).ApproxEqual(0.0) && Math.Log10(Convert.ToDouble(row["id:V_Wall"])) % 1  == 0.0);

In [ ]:
List<Plot2Ddata> pltsL = new List<Plot2Ddata>();
foreach(string col in Columns){
    int k = Columns.IndexOf(col);
    var plt = tab2pltL.ToPlot("id:delta", col, ColName_GroupSelection: new string[] { "id:V_Wall" });    
    plt.LogX   = true;
    plt.LogY   = true;
    plt.Xlabel = @"Nip Width $\\\left[m\\right]$";
    plt.Ylabel = Labels[k] + " " + Units[k];
    plt.LabelTitleFont = 32;
    plt.LabelFont = 24;
    plt.LegendFont = 18;
    plt.ShowLegend = true;    
    plt.LegendBox = true;
    
    plt.LegendAlignment = new string[]{"o", "c", "t"};
    plt.LegendSwap = true;  
    plt.LegendHorizontal = true;    

    plt.lmargin = 8;
    plt.rmargin = 1;
    plt.tmargin = 6;
    plt.bmargin = 1;

    plt.LogX2 = true;
    plt.LogY2 = true;

    for(int n= 0; n < plt.dataGroups.Count(); n++){
        plt.dataGroups[n].Format.LineWidth = 2;
        plt.dataGroups[n].Format.PointSize = 1;
        plt.dataGroups[n].Name = "$V_W = " + Convert.ToDouble(plt.dataGroups[n].Name.Split("id:V_Wall").Last()).ToString("N1") + @" \\\frac{m}{s}$";
    }
    plt.ModFormat();
    pltsL.Add(plt);
}

In [ ]:
var tab2pltR = tab.ExtractRows((i,row) => Convert.ToDouble(row["id:V_Wall"]).ApproxEqual(0.0) && new double[] {2, 6, 10}.Contains(Math.Log10(Convert.ToDouble(row["id:P_Diff"]))));
tab2pltR.DefaultView.Sort = "id:P_Diff DESC";
tab2pltR = tab2pltR.DefaultView.ToTable();

In [ ]:
List<Plot2Ddata> pltsR = new List<Plot2Ddata>();
foreach(string col in Columns){
    int k = Columns.IndexOf(col);
    var plt = tab2pltR.ToPlot("id:delta", col, ColName_GroupSelection: new string[] { "id:P_Diff" });    
    plt.LogX   = true;
    plt.LogY   = true;
    plt.Xlabel = @"Nip Width $\\\left[m\\right]$";    
    plt.Ylabel = Labels[k] + " " + Units[k];
    plt.LabelTitleFont = 32;
    plt.LabelFont = 24;
    plt.LegendFont = 18;
    plt.ShowLegend = true;    
    plt.LegendBox = true;
    
    plt.LegendAlignment = new string[]{"o", "c", "t"};
    plt.LegendSwap = true;    
    plt.LegendHorizontal = true;
    
    plt.lmargin = 6;
    plt.rmargin = 3;
    plt.tmargin = 6;
    plt.bmargin = 1;

    plt.LogX2 = true;
    plt.LogY2 = true;
   
    for(int n= 0; n < plt.dataGroups.Count(); n++){
        plt.dataGroups[n].Format.LineWidth = 2;
        plt.dataGroups[n].Format.PointSize = 1;
        plt.dataGroups[n].Name = @"$\\\Delta{p} = " + Convert.ToDouble(plt.dataGroups[n].Name.Split("id:P_Diff").Last()).ToString("0e0").Replace("e","\\\\times 10^{") + "}" + @" \\\frac{N}{m^2}$";
    }
    plt.ModFormat();
    pltsR.Add(plt);
}

In [ ]:
int k = 0;
foreach(var pltL in pltsL){
    var gp = new Gnuplot();
    string label = pltL.Ylabel;
    if(pltsR.Where(plt => plt.Ylabel == label).Count() > 0){
        var pltR = pltsR.SingleOrDefault(plt => plt.Ylabel == label) ;
        pltR.Ylabel = null;
        
        gp.Cmd("set multiplot layout 1,2 rowsfirst");
        gp.PlotLogSlope(pltL, format: new PlotFormat(lineColor: (LineColors)7));       
        gp.Cmd("set key box width -5");
        gp.Cmd("set size square");
        pltL.ToGnuplot(gp);

        gp.Cmd("unset label");
        gp.Cmd("unset object");

        gp.PlotLogSlope(pltR, format: new PlotFormat(lineColor: (LineColors)7));
        gp.Cmd("set key box width -7");
        gp.Cmd("set size square");
        pltR.ToGnuplot(gp);

        gp.Cmd("unset multiplot");

        gp.PlotCairolatex(ySize: 8.0).SaveTo("./PrintingNip/Figures/Part1/Multiplot_"+Columns[k]+".tex");
    }    
    k++;
}

### Test Plotting

In [ ]:
// Test code - see TestPDF.tex for output preview
// int i = 5;
// var gp = new Gnuplot();
// gp.PlotLogSlope(plts.Pick(1));
// gp.Cmd("set key box width -5");
// plts.Pick(i).ToGnuplot(gp);
// gp.PlotCairolatex().SaveTo("./PrintingNip/Figures/Test/Part1.tex");